In [72]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

In [73]:
# perf_dir = "/home/shtoshni/Research/events/models/perf"
perf_dir = "/share/data/speech/shtoshni/research/events/models/perf"
slurm_id_list = ["6441801"]  #"6407936"

files = []

for slurm_id in slurm_id_list:
    files.extend(sorted(glob.glob(path.join(perf_dir, slurm_id + "*")), 
                                  key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1])))
    
# files = sorted(files, key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1]))
print(len(files))

48


In [74]:
model_dict_list = []
for file in files:
    model_dict_list.append(json.loads(open(file).read()))

In [75]:
def determine_varying_attributes(model_dict_list, ignore_attribs=['model_dir', 'train', 'test', 'dev', 'pretrained_mention_model',
                                                                  'slurm_id', 'best_model_dir', 'data_dir']):
    attrib_to_vals = defaultdict(set)
    for model_dict in model_dict_list:
        for attrib, val in model_dict.items():
            if attrib in ignore_attribs:
                continue
            else:
                attrib_to_vals[attrib].add(val)
    
    
    varying_attribs = []
    for attrib, vals in attrib_to_vals.items():
        if len(vals) > 1:
            varying_attribs.append(attrib)
            
    return varying_attribs

In [76]:
varying_attribs = determine_varying_attributes(model_dict_list)


In [77]:
perf_df = pd.DataFrame(columns=(varying_attribs))

for model_dict in model_dict_list:
    perf_dict = {}
    for attrib in varying_attribs:
        perf_dict[attrib] = model_dict[attrib]
            
    perf_dict['fscore'] = round(model_dict['dev']['fscore'], 1)
    perf_dict['test_f'] = round(model_dict['test']['fscore'], 1)
                
    perf_df = perf_df.append(perf_dict, ignore_index=True)

In [81]:
idx = perf_df.groupby(varying_attribs)['fscore'].idxmax()
dev_max = perf_df.iloc[idx]
# dev_max

In [79]:
if 'seed' in varying_attribs:
    varying_attribs.remove('seed')
perf_df.groupby(varying_attribs)[['fscore', 'test_f']].mean()


fscore  test_f
rnn_size use_mem_context sample_invalid event_subtype_loss_wt                
500      False           0.2            0.01                    46.50   46.85
                                        0.10                    46.15   47.00
                                        1.00                    46.60   48.05
                         0.5            0.01                    45.45   48.30
                                        0.10                    45.80   47.55
                                        1.00                    45.50   46.95
         True            0.2            0.01                    47.80   47.80
                                        0.10                    48.20   47.20
                                        1.00                    46.30   47.60
                         0.5            0.01                    46.45   47.85
                                        0.10                    47.10   47.60
                                        1.00                    46.00   46.80
1000     False           0.2            0.01                    47.25   47.25
                                        0.10                    47.45   46.45
                                        1.00                    46.05   46.30
                         0.5            0.01                    46.45   47.90
                                        0.10                    46.05   47.20
                                        1.00                    47.45   46.90
         True            0.2            0.01                    46.50   48.40
                                        0.10                    46.20   46.75
                                        1.00                    46.50   46.80
                         0.5            0.01                    46.00   46.65
                                        0.10                    46.50   46.45
                                        1.00                    45.80   45.50

In [82]:
perf_df.groupby(['use_mem_context'])[['fscore', 'test_f']].mean()


,fscore,test_f
use_mem_context,,
False,46.391667,47.225000
True,46.612500,47.116667


In [83]:
perf_df.groupby(['use_mem_context'])[['fscore', 'test_f']].mean()


,fscore,test_f
use_mem_context,,
False,46.391667,47.225000
True,46.612500,47.116667


In [84]:
perf_df.groupby(['event_subtype_loss_wt'])[['fscore', 'test_f']].mean()


,fscore,test_f
event_subtype_loss_wt,,
0.01,46.55000,47.6250
0.10,46.68125,47.0250
1.00,46.27500,46.8625


In [85]:
perf_df.groupby(['sample_invalid'])[['fscore', 'test_f']].mean()


,fscore,test_f
sample_invalid,,
0.2,46.791667,47.204167
0.5,46.212500,47.137500


In [86]:
perf_df.groupby(['rnn_size'])[['fscore', 'test_f']].mean()

,fscore,test_f
rnn_size,,
500,46.487500,47.462500
1000,46.516667,46.879167
